In [ ]:
import sys, time # Computing time for each algorithm
import math # Doing mathematical computations
import random # Generating large random string
from IPython.display import Image, display # Displaying images

In [ ]:
IN_FILE = 'data/chrMT.fna'

In [ ]:
# Command line prompt for finding the length of input sequence
!seqkit fx2tab --length --name $IN_FILE

In [ ]:
length = 16569

In [ ]:
bit_vector = '0'
cnt = 0

# Bit vector will be used to support operations of the form: Finding number of character 'A' till index i

with open(IN_FILE, 'r') as file:
    for line in file:
        token = line.strip()
        if(token[0] == '>'):
            continue
        for c in token:
            cnt += 1
            bit_vector += '1' if c == 'A' else '0'  

assert cnt == length

In [ ]:
# Naive Algorithm

# Implementation 

naive_rank = [0] * (length + 1)

for i in range(1, length + 1):
    naive_rank[i] = naive_rank[i - 1] + (0 if bit_vector[i] == '0' else 1) # Using Prefix sum technique to compute the rank at every index

n_space = sys.getsizeof(naive_rank) / 1000000
print(f'Space Required by Naive Algorithm = {n_space} MB')

In [ ]:
# Succinct Algorithm 

# Implementation 

k = math.floor(math.log2(length) / 2)
l = k * k

first = [0] * (int(length / l) + 2)
second = [0] * (int(length / k) + 2)

for i in range(1, length + 1):
    first[math.ceil(i / l)] += 0 if bit_vector[i] == '0' else 1
    second[math.ceil(i / k)] += 0 if bit_vector[i] == '0' else 1
    
    if(i % k == 0):
        second[int(i / k)] += second[int(i / k) - 1]
    if(i % l == 0):
        first[int(i / l)] += first[int(i / l) - 1]
        second[int(i / k)] = 0

third = []

for i in range(2 ** (k - 1)):
    pthird = [0] * k
    for j in range(k - 1): # 0110 & 1 = 0 -> 0011 & 1 = 1 -> 1001 & 1 = 1 -> 1100 & 1 = 0 -> 0110
        pthird[j + 1] = pthird[j] + ((i >> (k - 2 - j)) & 1)
    third.append(pthird)

s_space = (sys.getsizeof(bit_vector) + sys.getsizeof(first) + sys.getsizeof(second) + sys.getsizeof(third)) / 1000000

print(f'Space required by the Succinct Algorithm = {s_space} MB')

print(f'Succinct Algorithm takes {n_space / s_space} x less space in comparison to Naive Algorithm')

In [ ]:
# Stress test + Timing the Succinct Algorithm

# Pad bit_vector with k zeroes
for i in range(k):
    bit_vector += '0'

query_count = 1000000
naive_total_time = 0
succinct_total_time = 0

for i in range(query_count):
    id = random.randint(1, length)
    
    naive_starttime = time.time()
    n_rank = naive_rank[id]
    naive_endtime = time.time()

    succinct_starttime = time.time()
    s_rank = first[int(id / l)] + second[int(id / k)] + third[int(bit_vector[int(id / k) * k + 1 : (int(id / k) + 1) * k], 2)][id % k]
    succinct_endtime = time.time()

    assert n_rank == s_rank

    naive_total_time += naive_endtime - naive_starttime
    succinct_total_time += succinct_endtime - succinct_starttime

print(f'Succinct Algorithm takes {succinct_total_time / naive_total_time}x more time w.r.t. the Naive Algorithm')